In [1]:
# Setup and Imports
import time
import datetime
import requests
import hashlib
from pow.app.client_v1 import ClientV1
from pow.compute.stats import estimate_R_from_experiment
from pow.compute.compute import ProofBatch
from pow.data import ValidatedBatch
from pow.models.utils import Params

# Generate unique identifiers for this session
date_str = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
BLOCK_HASH = hashlib.sha256(date_str.encode()).hexdigest()
PUBLIC_KEY = f"pub_key_1_{date_str}"
BATCH_SIZE = 5000

In [2]:
# API Configuration
client_url = "http://172.18.116.100:8081" #batch reciever
server_url = "http://172.18.116.100:8080"
client = ClientV1(server_url)
BASE_URL = client_url

In [3]:
# Helper Functions for API Interaction
def get_proof_batches():
    """Fetch all generated proof batches from the server"""
    response = requests.get(f"{BASE_URL}/generated")
    if response.status_code == 200:
        return response.json()["proof_batches"]
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

def get_val_proof_batches():
    """Fetch all validated proof batches from the server"""
    response = requests.get(f"{BASE_URL}/validated")
    if response.status_code == 200:
        return response.json()["validated_batches"]
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

# Calculate R estimate for the experiment
R_ESTIMATE = estimate_R_from_experiment(
    n=8192,
    P=0.001,
    num_samples=50000
)
R_TARGET = R_ESTIMATE

# Define model parameters
params = Params(
    dim=512,
    n_layers=64,
    n_heads=128,
    n_kv_heads=128,
    vocab_size=8192,
    ffn_dim_multiplier=16.0,
    multiple_of=1024,
    norm_eps=1e-05,
    rope_theta=500000.0,
    use_scaled_rope=False,
    seq_len=4
)

# Set fraud detection threshold
fraud_threshold = 0.01

# Utility functions for testing
def create_correct_batch(pb, n=10000):
    """Create a batch of correct proofs for testing"""
    correct_pb_dict = {
        'public_key': pb.public_key,
        'block_hash': pb.block_hash,
        'block_height': pb.block_height,
        'nonces': [pb.nonces[0]] * n,
        'dist': [pb.dist[0]] * n
    }
    return ProofBatch(**correct_pb_dict)

def get_incorrect_nonce(pb):
    """Generate an invalid nonce for testing"""
    for i in range(1000):
        if i not in pb.nonces:
            return i
    return None

def create_incorrect_batch(pb, n, n_invalid):
    """Create a batch with some invalid proofs for testing"""
    incorrect_pb_dict = {
        'public_key': pb.public_key,
        'block_hash': pb.block_hash,
        'block_height': pb.block_height,
        'nonces': [get_incorrect_nonce(pb)] * n_invalid,
        'dist': [pb.dist[0]] * n_invalid
    }
    return ProofBatch.merge([
        create_correct_batch(pb, n-n_invalid), 
        ProofBatch(**incorrect_pb_dict)
    ])

In [4]:
# Example Usage:
# 1. Initialize proof generation
client.init_generate(
    url=client_url,
    block_hash=BLOCK_HASH,
    block_height=1,
    public_key=PUBLIC_KEY,
    batch_size=BATCH_SIZE,
    r_target=R_TARGET,
    fraud_threshold=fraud_threshold,
    params=params,
)
# Wait for the server to generate some batches (about a minute)

{'status': 'OK',
 'pow_status': {'status': 'GENERATING', 'is_model_initialized': False}}

In [5]:
# 2. Get generated proofs
proof_batches = get_proof_batches()
pb = ProofBatch(**proof_batches[-1])

In [6]:
# 3. Create test batches
incorrect_pb = create_incorrect_batch(pb, 2000, 10)  # 2000 total proofs, 10 invalid
correct_pb = create_correct_batch(pb, 2000)          # 2000 valid proofs

In [7]:
# 4. Start validation
client.start_validation()
# alternative way, which removes the model and put it back.
# client.stop
# client.init_validate(...)

{'status': 'OK',
 'pow_status': {'status': 'VALIDATING', 'is_model_initialized': True}}

In [8]:
# 5. Submit batches for validation
client.validate(correct_pb)

In [9]:
client.validate(incorrect_pb)

In [10]:
# 6. Check validation results
val_proof_batches = get_val_proof_batches()

In [11]:
# Print results for both batches
vpb = ValidatedBatch(**val_proof_batches[-2])  # Valid batch
print(f"Valid batch: size={len(vpb)}, invalid={vpb.n_invalid}, p_honest={vpb.probability_honest:.2e}, "
      f"threshold={vpb.fraud_threshold:.2e}, fraud={vpb.fraud_detected}")

vpb = ValidatedBatch(**val_proof_batches[-1])  # Invalid batch
print(f"Invalid batch: size={len(vpb)}, invalid={vpb.n_invalid}, p_honest={vpb.probability_honest:.2e}, "
      f"threshold={vpb.fraud_threshold:.2e}, fraud={vpb.fraud_detected}")

Valid batch: size=2000, invalid=0, p_honest=1.00e+00, threshold=1.00e-02, fraud=False
Invalid batch: size=2000, invalid=10, p_honest=1.09e-07, threshold=1.00e-02, fraud=True
